In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from vis3d_utilities import *
from metadata import *
from data_manager import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Local Macbook Pro


In [6]:
brain_spec = dict(name='atlasV5', 
                  resolution='10.0um', 
                  vol_type='score', 
                  prep_id=None,
                  detector_id=None,
                  structure=None)

In [3]:
canonical_centroid_rel2uncropped = np.loadtxt(DataManager.get_atlas_canonical_centroid_filepath(atlas_name))

In [ ]:
structure_meshes_rel2canon = DataManager.load_meshes_v2(stack_m=atlas_name, sided=True, include_surround=False)